### [Линк](https://colab.research.google.com/drive/1mguVQuMEn2mIfISPCf4I9P6rvjNAK2ub?usp=sharing) до самиот Notebook

### Вовед

#### Import на библиотеките кои се користат во кодот

In [4]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report, f1_score, log_loss, precision_score, recall_score

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile

np.random.seed(1)

#### Симнување на glove векторите за репрезентација на зборови

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

with zipfile.ZipFile("glove.6B.zip","r") as zip_ref:
    zip_ref.extractall(".")

### Помошни функции

#### Исчитување на glove фајлот со вредности

In [6]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
            
    return words_to_index, index_to_words, word_to_vec_map

##### Softmax функцијата

In [7]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    
    return e_x / e_x.sum()

##### Читање на соодветните вредности од CSV train и test датотеките

In [8]:
def read_csv(filename):
    phrase = []
    emoji_ = []

    with open(filename) as csvDataFile:
        csv_reader = csv.reader(csvDataFile)

        for row in csv_reader:
            phrase.append(row[0])
            emoji_.append(row[1])

    x = np.asarray(phrase)
    y = np.asarray(emoji_, dtype=int)

    return x, y


##### Излезите(бројки) ги претвора во one-hot вектори

In [9]:
def convert_to_one_hot(y, c):
    y = np.eye(c)[y.reshape(-1)]
    
    return y

##### Предвидување на излезите при дадени елементи како влез

In [10]:
def predict(X, Y, W, b, word_to_vec_map):
    m = X.shape[0]
    pred = np.zeros((m, 1))

    for j in range(m):
        words = X[j].lower().split()

        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg / len(words)

        z = np.dot(W, avg) + b
        a = softmax(z)
        pred[j] = np.argmax(a)

    print("Accuracy: " + str(np.mean((pred[:] == Y.reshape(Y.shape[0], 1)[:]))))

    return pred

##### Претворање на реченици дадени како влез во матрици од вредности

In [11]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0] 
    x_indices = np.zeros((m, max_len))

    for i in range(m):
        sentence_words = (X[i].lower()).split()
        j = 0
    
        for w in sentence_words:
            # i-тата вредност е редниот број на реченицата, j-тата е редниот
            # број на зборот во неа. вредноста која се поставува е таа на 
            # соодветниот збор
            x_indices[i, j] = word_to_index[w]
            j = j + 1
    
    return x_indices

##### Креирање на веќе истрениран Embedding слој со помош на glove векторите

In [12]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    emb_matrix = np.zeros((vocab_len, emb_dim))

    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

##### Креирање на модел со соодветни предефинирани вредности

In [13]:
def sentiment_analysis(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape=input_shape, dtype=np.int32)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)

    # LSTM слој со 128-димензионален hidden state
    X = LSTM(128, return_sequences=True)(embeddings)
    
    # Веројатност на зачувување од 0.5
    X = Dropout(0.5)(X)
    
    # Уште еден LSTM слој со 128-димензионален hidden state
    X = LSTM(128)(X)

    X = Dropout(0.5)(X)
    X = Dense(5, activation='softmax')(X)
    X = Activation('softmax')(X)

    model = Model(sentence_indices, X)

    return model

# Main дел

##### Читање на train и test податоците, пренос на излезите како one-hot вектори, читање на векторите за репрезентација на зборови

In [14]:
X_train, Y_train = read_csv('train_set.csv')
X_test, Y_test = read_csv('test_set.csv')

maxLen = len(max(X_train, key=len).split())

Y_oh_train = convert_to_one_hot(Y_train, 5)
Y_oh_test = convert_to_one_hot(Y_test, 5)

word_to_index_, index_to_word, word_to_vec_map_ = read_glove_vecs('glove.6B.50d.txt')

##### Креирање и тренирање на моделот

In [15]:
model = sentiment_analysis((maxLen,), word_to_vec_map_, word_to_index_)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train_indices = sentences_to_indices(X_train, word_to_index_, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, 5)

model.fit(X_train_indices, Y_train_oh, epochs=100, batch_size=32, shuffle=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645   

5/5 [==============================] - 1s 293ms/step - loss: 0.9092 - accuracy: 0.9962
Epoch 69/100
5/5 [==============================] - 2s 305ms/step - loss: 0.9093 - accuracy: 0.9962
Epoch 70/100
5/5 [==============================] - 1s 264ms/step - loss: 0.9091 - accuracy: 0.9962
Epoch 71/100
5/5 [==============================] - 2s 327ms/step - loss: 0.9106 - accuracy: 0.9944
Epoch 72/100
5/5 [==============================] - 1s 279ms/step - loss: 0.9135 - accuracy: 0.9918
Epoch 73/100
5/5 [==============================] - 2s 305ms/step - loss: 0.9106 - accuracy: 0.9944
Epoch 74/100
5/5 [==============================] - 2s 301ms/step - loss: 0.9088 - accuracy: 0.9962
Epoch 75/100
5/5 [==============================] - 1s 268ms/step - loss: 0.9108 - accuracy: 0.9944
Epoch 76/100
5/5 [==============================] - 1s 266ms/step - loss: 0.9181 - accuracy: 0.9866
Epoch 77/100
5/5 [==============================] - 1s 297ms/step - loss: 0.9134 - accuracy: 0.9918
Epoch 78/100


##### Тестирање на моделот со test податоците

In [16]:
X_test_indices = sentences_to_indices(X_test, word_to_index_, max_len=maxLen)
Y_test_oh = convert_to_one_hot(Y_test, 5)

loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Accuracy ", acc)

2/2 [==============================] - 1s 6ms/step - loss: 1.1768 - accuracy: 0.7321

Accuracy  0.7321428656578064


##### Преглед на влезовите кои се грешно предвидени и дополнителни метрики за евалуација

In [17]:
y_test_oh = np.eye(5)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index_, maxLen)
pred = model.predict(X_test_indices)

actual = []
predicted = []

for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])

    actual.append(Y_test[i])
    predicted.append(num)

    if num != Y_test[i]:
        print('Input: ' + str(X_test[i]))
        print('Expected class: ' + str(Y_test[i]))
        print('Predicted class: ' + str(num) + '\n')

precision = precision_score(actual, predicted, average='macro')
recall = recall_score(actual, predicted, average='macro')
f1_score = f1_score(actual, predicted, average='macro')
loss = log_loss(actual, pred, eps=1e-15)
matrix = classification_report(actual, predicted, labels=[0, 1, 2, 3, 4])

Input: he got a very nice raise
Expected class: 2
Predicted class: 0

Input: she got me a nice present
Expected class: 2
Predicted class: 0

Input: he is a good friend
Expected class: 2
Predicted class: 0

Input: I am upset
Expected class: 3
Predicted class: 1

Input: work is hard
Expected class: 3
Predicted class: 2

Input: This girl is messing with me
Expected class: 3
Predicted class: 0

Input: any suggestions for dinner
Expected class: 4
Predicted class: 2

Input: you brighten my day
Expected class: 2
Predicted class: 0

Input: she is a bully
Expected class: 3
Predicted class: 0

Input: I am upset
Expected class: 3
Predicted class: 1

Input: My life is so boring
Expected class: 3
Predicted class: 0

Input: will you be my valentine
Expected class: 2
Predicted class: 0

Input: What you did was awesome
Expected class: 2
Predicted class: 0

Input: I do not want to joke
Expected class: 3
Predicted class: 0

Input: go away
Expected class: 3
Predicted class: 1



##### Тестирање на моделот со влезови од корисник

In [18]:
x_test = np.array(['very happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index_, maxLen)
print('Input: ' + x_test[0])
print('Predicted class: ' + str(np.argmax(model.predict(X_test_indices))) + '\n')

Input: very happy
Predicted class: 2



In [19]:
x_test = np.array(['very sad'])
X_test_indices = sentences_to_indices(x_test, word_to_index_, maxLen)
print('Input: ' + x_test[0])
print('Predicted class: ' + str(np.argmax(model.predict(X_test_indices))) + '\n')

Input: very sad
Predicted class: 2



In [20]:
x_test = np.array(['i am starving'])
X_test_indices = sentences_to_indices(x_test, word_to_index_, maxLen)
print('Input: ' + x_test[0])
print('Predicted class: ' + str(np.argmax(model.predict(X_test_indices))) + '\n')

Input: i am starving
Predicted class: 4



In [21]:
x_test = np.array(['I have met the love of my life'])
X_test_indices = sentences_to_indices(x_test, word_to_index_, maxLen)
print('Input: ' + x_test[0])
print('Predicted class: ' + str(np.argmax(model.predict(X_test_indices))) + '\n')

Input: I have met the love of my life
Predicted class: 0



##### Приказ на мерките за успешност на моделот

In [22]:
print('Accuracy: {0}'.format(str(acc)))

Accuracy: 0.7321428656578064


In [23]:
print('Log loss: {0}'.format(loss))

Log loss: 1.176811202296189


In [24]:
print('Precision: {0}'.format(precision))

Precision: 0.7992360580595875


In [25]:
print('Recall: {0}'.format(recall))

Recall: 0.8047619047619048


In [26]:
print('F1 score: {0}'.format(f1_score))

F1 score: 0.7530364372469636


In [27]:
print('Classification report: \n{0}'.format(matrix))

Classification report: 
              precision    recall  f1-score   support

           0       0.41      1.00      0.58         7
           1       0.73      1.00      0.84         8
           2       0.86      0.67      0.75        18
           3       1.00      0.50      0.67        16
           4       1.00      0.86      0.92         7

    accuracy                           0.73        56
   macro avg       0.80      0.80      0.75        56
weighted avg       0.84      0.73      0.74        56

